In [1]:
pip install ipytest

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pytest-mock

Note: you may need to restart the kernel to use updated packages.


In [248]:
import pytest
import ipytest
import os

ipytest.autoconfig(addopts=["-v"])

# Задание 1

In [249]:
%%ipytest
def is_palindrome(string):
    try:
        reversed_string = ""

        # Удалим все пробелы
        s = string.replace(" ","")
        # переведем в нижний реестр
        s = s.lower()
        for i in range(len(s), 0, -1):
            reversed_string += s[i - 1]


        if (s == reversed_string) and (string.strip()) and (len(s)>1) and (isinstance(s, str)):
            return True
        else:
            return False
    except Exception:
        return False

@pytest.mark.parametrize("s,expected", [("@а л л а@", True), ("Ser", False), ("bob", True), (",_,_,", True)])
def test_is_palindrome(s, expected):
    assert is_palindrome(s)==expected
    
def test_palindrome_empty_string():
    assert is_palindrome("     ") == False

def test_palindrome_one_char():
    assert is_palindrome("t") == False    
    
def test_palindrome_is_str():
    assert is_palindrome(555) == False

    
 #тест класса проигрываний.   
    

======================================= test session starts =======================================
platform win32 -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0 -- E:\Anaconda\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Wisk
plugins: anyio-3.5.0, mock-3.10.0
collecting ... collected 7 items

tmphue_7zx3.py::test_is_palindrome[@\u0430 \u043b \u043b \u0430@-True] PASSED                [ 14%]
tmphue_7zx3.py::test_is_palindrome[Ser-False] PASSED                                         [ 28%]
tmphue_7zx3.py::test_is_palindrome[bob-True] PASSED                                          [ 42%]
tmphue_7zx3.py::test_is_palindrome[,_,_,-True] PASSED                                        [ 57%]
tmphue_7zx3.py::test_palindrome_empty_string PASSED                                          [ 71%]
tmphue_7zx3.py::test_palindrome_one_char PASSED                                              [ 85%]
tmphue_7zx3.py::test_palindrome_is_str PASSED                                                [100%]

# Задание 2

In [250]:
DEVELOP_NAME = "Bob"
DESIGN_NAME = "Anna"
DESIGN_AWARDS = 0

In [251]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

In [252]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [253]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards
        self.seniority += awards*2
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        #print('количество баллов', self.seniority)
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [254]:
@pytest.fixture
def designer():
    def _designer(seniority):
        return Designer(name=DESIGN_NAME,seniority=seniority,awards=DESIGN_AWARDS)
    return _designer

@pytest.fixture
def developer():
    def _developer(seniority):
        return Developer(name=DEVELOP_NAME,seniority=seniority)
    return _developer

In [255]:
%%ipytest
@pytest.mark.parametrize("seniority_designer,real_grade_des,seniority_developer,real_grade_dev",[(2,4,4,5),(4,4,8,5)])
def test_des_dev(designer,developer,seniority_designer,real_grade_des,seniority_developer,real_grade_dev):
    a = designer(seniority_designer)
    b = developer(seniority_developer)
    for i in range(20):
        b.check_if_it_is_time_for_upgrade()
        a.check_if_it_is_time_for_upgrade()
    assert b.grade == real_grade_dev
    assert a.grade == real_grade_des

======================================= test session starts =======================================
platform win32 -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0 -- E:\Anaconda\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Wisk
plugins: anyio-3.5.0, mock-3.10.0
collecting ... collected 2 items

tmpmph8ac7i.py::test_des_dev[2-4-4-5] PASSED                                                 [ 50%]
tmpmph8ac7i.py::test_des_dev[4-4-8-5] PASSED                                                 [100%]

======================================== 2 passed in 0.04s ========================================


# Задание 3

In [182]:
import requests
import json
from unittest.mock import Mock, MagicMock, patch

In [256]:
class Rate:
    def __init__(self, format='value',diff=False):
        self.format = format
        self.diff=diff
       
    def max_cur(self):
        r_json = self.exchange_rates()
        all_currencies_value={}
        for element in r_json:
            all_currencies_value[element]=r_json[element]['Value']
        max_currencies=max(all_currencies_value.keys(),key=(lambda x:all_currencies_value[x]))
        return(r_json[max_currencies]['Name'])

    def exchange_rates(self):
        r = requests.get("https://www.cbr-xml-daily.ru/daily_json.js")
        return r.json()['Valute']
    
    def make_format(self, currency):
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                
                if not self.diff:
                    return response[currency]['Value']
                else:
                    return response[currency]['Value']-response[currency]['Previous']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [239]:
a=Rate(format="full",diff=False)

In [246]:
a.make_format("AUD")

{'ID': 'R01010',
 'NumCode': '036',
 'CharCode': 'AUD',
 'Nominal': 1,
 'Name': 'Австралийский доллар',
 'Value': 39.8703,
 'Previous': 39.3121}

In [247]:
%%ipytest

@pytest.mark.parametrize("valute,format_v,diff_v,expected",[("AUD","value",False, 39.8703),("AZN","value",True, -0.6032000000000011),
("AZN","full",True, "{'ID': 'R01020A',  'NumCode': '944',  'CharCode': 'AZN',  'Nominal': 1,  'Name': 'Азербайджанский манат',  'Value': 35.4223,  'Previous': 36.0255}"),
("AUD","full",False, "{'ID': 'R01010',  'NumCode': '036',  'CharCode': 'AUD',  'Nominal': 1,  'Name': 'Австралийский доллар',  'Value': 39.8703,  'Previous': 39.3121}"),])
def test_make_format(mocker,valute,format_v,diff_v,expected):
    str_json = '''{
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 39.8703,
            "Previous": 39.3121
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 35.4223,
            "Previous": 36.0255
        }
    }'''
    mocker.patch(__name__ + ".Rate.exchange_rates", return_value=json.loads(str_json))
    c = Rate(format=format_v,diff=diff_v)
    if format_v == "value":
        assert c.make_format(valute) == expected
    else:
        valid_json = expected.replace("\'", "\"")
        str_expected = json.loads(valid_json)
        assert c.make_format(valute) == str_expected


======================================= test session starts =======================================
platform win32 -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0 -- E:\Anaconda\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Wisk
plugins: anyio-3.5.0, mock-3.10.0
collecting ... collected 4 items

tmpkwkb9z5j.py::test_make_format[AUD-value-False-39.8703] PASSED                             [ 25%]
tmpkwkb9z5j.py::test_make_format[AZN-value-True--0.6032000000000011] PASSED                  [ 50%]
tmpkwkb9z5j.py::test_make_format[AZN-full-True-{'ID': 'R01020A',  'NumCode': '944',  'CharCode': 'AZN',  'Nominal': 1,  'Name': '\u0410\u0437\u0435\u0440\u0431\u0430\u0439\u0434\u0436\u0430\u043d\u0441\u043a\u0438\u0439 \u043c\u0430\u043d\u0430\u0442',  'Value': 35.4223,  'Previous': 36.0255}] PASSED [ 75%]
tmpkwkb9z5j.py::test_make_format[AUD-full-False-{'ID': 'R01010',  'NumCode': '036',  'CharCode': 'AUD',  'Nominal': 1,  'Name': '\u0410\u0432\u0441\u0442\u0440\u0430\u043b\u0438\u0439\u0441\